In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Clustering

Let's start by creating a simple, 2-dimensional, synthetic dataset:

In [ ]:
from sklearn.datasets import make_blobs
random_state = 42

X, y = make_blobs(random_state=random_state)
X.shape


In [ ]:
plt.scatter(X[:, 0], X[:, 1]);


* In the scatter plot above, we can see three separate groups of data points and we would like to recover them using clustering.
* Think of "discovering" the class labels that we already take for granted in a classification task.
* Even if the groups are obvious in the data, it is hard to find them when the data lives in a high-dimensional space, which we can't visualize in a single histogram or scatterplot.

## K-means
* Now we will use one of the simplest clustering algorithms, K-means.
* This is an iterative algorithm which searches for three cluster centers such that the distance from each point to its cluster is minimized.
* The standard implementation of K-means uses the Euclidean distance.

**Question:** what would you expect the output to look like?

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=random_state)


* We can get the cluster labels either by calling fit and then accessing the ``labels_`` attribute of the K means estimator, or by calling ``fit_predict``.
* Either way, the result contains the ID of the cluster that each point is assigned to.

In [ ]:
labels = kmeans.fit_predict(X)


In [ ]:
labels


In [ ]:
all(y == labels)


Let's visualize the assignments that have been found

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels);


Compared to the true labels:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y);


* Here, we are probably satisfied with the clustering results.
* But in general we might want to have a more quantitative evaluation.
* How about comparing our cluster labels with the ground truth we got when generating the blobs?

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

print('Accuracy score:', accuracy_score(y, labels))
print(confusion_matrix(y, labels))


In [ ]:
np.mean(y == labels)


**Exercise**

After looking at the "True" label array y, and the scatterplot and `labels` above, can you figure out why our computed accuracy is 0.0, not 1.0, and can you fix it?

* Even though we recovered the partitioning of the data into clusters perfectly, the cluster IDs we assigned were arbitrary, and we can not hope to recover them.
* Therefore, we must use a different scoring metric, such as ``adjusted_rand_score``, which is invariant to permutations of the labels:

In [ ]:
from sklearn.metrics import adjusted_rand_score

adjusted_rand_score(y, labels)


* One of the "short-comings" of K-means is that we have to specify the number of clusters, which we often don't know *apriori*.
* For example, let's have a look what happens if we set the number of clusters to 2 in our synthetic 3-blob dataset:

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
labels = kmeans.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels);


## The Elbow Method

* The Elbow method is a "rule-of-thumb" approach to finding the optimal number of clusters.
* Here, we look at the cluster dispersion for different values of k:

In [ ]:
distortions = []
for i in range(1, 11):
    km = KMeans(n_clusters=i, 
                random_state=0)
    km.fit(X)
    distortions.append(km.inertia_)
plt.plot(range(1, 11), distortions, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

plt.show()


Then, we pick the value that resembles the "pit of an elbow." As we can see, this would be k=3 in this case, which makes sense given our visual expection of the dataset previously.

**Clustering comes with assumptions**:
* A clustering algorithm finds clusters by making assumptions with samples should be grouped together.
* Each algorithm makes different assumptions and the quality and interpretability of your results will depend on whether the assumptions are satisfied for your goal.
* For K-means clustering, the model is that all clusters have equal, spherical variance.

**In general, there is no guarantee that structure found by a clustering algorithm has anything to do with what you were interested in**.


We can easily create a dataset that has non-isotropic clusters, on which kmeans will fail:

In [ ]:
# Clustering of blobs
n_samples = 1500
random_state = 170

X, y = make_blobs(n_samples=n_samples, random_state=random_state)

kmeans = KMeans(n_clusters=2, random_state=random_state)

y_pred = kmeans.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=y_pred);
plt.title("Correct clustering");

#try to change number of clusters


In [ ]:
# Different variance
n_samples = 1500

X_varied, y_varied = make_blobs(n_samples=n_samples,
                                cluster_std=[1.0, 2.5, 0.5],
                                random_state=random_state)

kmeans = KMeans(n_clusters=3, random_state=random_state)

y_pred = kmeans.fit_predict(X_varied)


plt.scatter(X_varied[:, 0], X_varied[:, 1], c=y_pred);
plt.title("Unequal Variance");


### Unevenly sized blobs

In [ ]:
X, y = make_blobs(n_samples=n_samples,
                  cluster_std=[2.0, 2.0, 2.0],
                  random_state=random_state)
X_filtered = np.vstack((X[y == 0][:500], X[y == 1][:100], X[y == 2][:10]))

kmeans = KMeans(n_clusters=3, random_state=random_state)

y_pred = kmeans.fit_predict(X_filtered)


plt.scatter(X_filtered[:, 0], X_filtered[:, 1], c=y_pred);
plt.title("Unevenly Sized Blobs");


### Anisotropically distributed data

In [ ]:
X, y = make_blobs(n_samples=n_samples,
                  cluster_std=[1.0, 1.0, 1.0],
                  random_state=random_state)

transformation = [[ 0.60834549, -0.63667341], [-0.40887718, 0.85253229]]
X_aniso = np.dot(X, transformation)

kmeans = KMeans(n_clusters=3, random_state=random_state)

y_pred = kmeans.fit_predict(X_aniso)

plt.scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred);
plt.title("Anisotropicly Distributed Blobs");


### GMM for anysotropic clusters

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=random_state)

gmm.fit(X_aniso)

y_pred = gmm.predict(X_aniso)

plt.scatter(X_aniso[:, 0], X_aniso[:, 1], c=y_pred);
plt.title("Anisotropicly Distributed Blobs");


## Some Notable Clustering Routines

The following are two well-known clustering algorithms. 

- `sklearn.cluster.KMeans`: <br/>
    The simplest, yet effective clustering algorithm. Needs to be provided with the
    number of clusters in advance, and assumes that the data is normalized as input
    (but use a PCA model as preprocessor).
- `sklearn.cluster.MeanShift`: <br/>
    Can find better looking clusters than KMeans but is not scalable to high number of samples.
- `sklearn.cluster.DBSCAN`: <br/>
    Can detect irregularly shaped clusters based on density, i.e. sparse regions in
    the input space are likely to become inter-cluster boundaries. Can also detect
    outliers (samples that are not part of a cluster).
- `sklearn.cluster.AffinityPropagation`: <br/>
    Clustering algorithm based on message passing between data points.
- `sklearn.cluster.SpectralClustering`: <br/>
    KMeans applied to a projection of the normalized graph Laplacian: finds
    normalized graph cuts if the affinity matrix is interpreted as an adjacency matrix of a graph.
- `sklearn.cluster.Ward`: <br/>
    Ward implements hierarchical clustering based on the Ward algorithm,
    a variance-minimizing approach. At each step, it minimizes the sum of
    squared differences within all clusters (inertia criterion).

Of these, Ward, SpectralClustering, DBSCAN and Affinity propagation can also work with precomputed similarity matrices.

<img src="figures/cluster_comparison.png" width="900">

## Exercise: digits clustering

* Perform K-means clustering on the digits data, searching for ten clusters.
* Visualize the cluster centers as images (i.e. reshape each to 8x8 and use ``plt.imshow``)
* Do the clusters seem to be correlated with particular digits?
* What is the ``adjusted_rand_score``?


In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()


In [ ]:
fig = plt.figure(figsize=(8, 3))

for i in range(10):
    ax = fig.add_subplot(2, 5, 1 + i)
    ax.imshow(kmeans.cluster_centers_[i].reshape((8, 8)), cmap=plt.cm.binary)
    

# Hierarchical Clustering

* One nice feature of hierachical clustering is that we can visualize the results as a dendrogram, a hierachical tree.
* Using the visualization, we can then decide how "deep" we want to cluster the dataset by setting a "depth" threshold
* Or in other words, we don't need to make a decision about the number of clusters upfront.


## Agglomerative and divisive hierarchical clustering

* Furthermore, we can distinguish between 2 main approaches to hierarchical clustering: Divisive clustering and agglomerative clustering.
* In agglomerative clustering, we start with a single sample from our dataset and iteratively merge it with other samples to form clusters - we can see it as a bottom-up approach for building the clustering dendrogram.  
* In divisive clustering, however, we start with the whole dataset as one cluster, and we iteratively split it into smaller subclusters - a top-down approach.  

In this notebook, we will use **agglomerative** clustering.

### Single and complete linkage

* Now, the next question is how we measure the similarity between samples.
* One approach is the familiar Euclidean distance metric that we already used via the K-Means algorithm. 
* As a refresher, the distance between 2 m-dimensional vectors $\mathbf{p}$ and $\mathbf{q}$ can be computed as:

\begin{align} \mathrm{d}(\mathbf{q},\mathbf{p}) & = \sqrt{(q_1-p_1)^2 + (q_2-p_2)^2 + \cdots + (q_m-p_m)^2} \\[8pt]
& = \sqrt{\sum_{j=1}^m (q_j-p_j)^2}.\end{align}	

* However, that's the distance between 2 samples.
* Now, how do we compute the similarity between subclusters of samples?
* I.e., our goal is to iteratively merge the most similar pairs of clusters until only one big cluster remains.
* There are many different approaches to this, for example single and complete linkage.

* In single linkage, we take the pair of the most similar samples (based on the Euclidean distance, for example) in each cluster, and merge the two clusters which have the most similar 2 members into one new, bigger cluster.
* In complete linkage, we compare the pairs of the two most dissimilar members of each cluster with each other, and we merge the 2 clusters where the distance between its 2 most dissimilar members is smallest.

![](figures/clustering-linkage.png)

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
n_samples, n_features = X.shape

plt.scatter(X[:, 0], X[:, 1], c=y);


In [ ]:
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram

clusters = linkage(X, 
                   metric='euclidean',
                   method='complete')

dendr = dendrogram(clusters)

plt.ylabel('Euclidean Distance');


In [ ]:
from sklearn.cluster import AgglomerativeClustering

ac = AgglomerativeClustering(n_clusters=3,
                             affinity='euclidean',
                             linkage='complete')

prediction = ac.fit_predict(X)
print('Cluster labels: %s\n' % prediction)


In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=prediction);


## Density-based Clustering - DBSCAN

* Another useful approach to clustering is *Density-based Spatial Clustering of Applications with Noise* (DBSCAN).
* In essence, we can think of DBSCAN as an algorithm that divides the dataset into subgroup based on dense regions of points.

In DBSCAN, we distinguish between 3 different "points":

- Core points: A core point is a point that has at least a minimum number of other points (MinPts) in its radius epsilon.
- Border points: A border point is a point that is not a core point, since it doesn't have enough MinPts in its neighborhood, but lies within the radius epsilon of a core point.
- Noise points: All other points that are neither core points nor border points.

![](figures/dbscan.png)

A nice feature about DBSCAN is that we don't have to specify a number of clusters upfront. However, it requires the setting of additional hyperparameters such as the value for MinPts and the radius epsilon.

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=400,
                  noise=0.1,
                  random_state=1)
plt.scatter(X[:,0], X[:,1]);
plt.show();


In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.2,
            min_samples=10,
            metric='euclidean')
prediction = db.fit_predict(X)

print("Predicted labels:\n", prediction)


In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=prediction);


## Exercise

* Using the following toy datasets, two concentric circles, experiment with the three different clustering algorithms that we used so far `KMeans`, `AgglomerativeClustering`, and `DBSCAN`.
* Which clustering algorithms reproduces or discovers the hidden structure (pretending we don't know `y`) best?
* Can you explain why this particular algorithm is a good choice while the other 2 "fail"?

In [ ]:
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=500, 
                    factor=.6, 
                    noise=.05)

plt.scatter(X[:, 0], X[:, 1], c=y);
